In [5]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import pandas as pd
import os
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By 

In [6]:
start_url = "https://www.naukri.com/{i}-jobs?k={i}"

In [7]:
def get_url_data(url, path):
    global file_name_count
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="root"]/div[3]/div[4]/div/button').click()
    soup = BeautifulSoup(driver.page_source,"html.parser")
    results = soup.find(class_="list")
    job_elems = results.find_all("article",class_="jobTuple bgWhite br4 mb-8")
    c = 1
    for element in job_elems:
        window_before = driver.window_handles[0]
        try:
            click_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[3]/div[2]/section[2]/div[2]/article[{0}]'.format(c))))
        
            click_button.click()
            window_after = driver.window_handles[1]
            driver.switch_to_window(window_after)
            try:
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,"html.parser")
                data_to_txt_file = {}
                try:
                    job_description = soup.find(class_="dang-inner-html")
                    data_to_txt_file["JOB DESCRIPTION"] = job_description.get_text()
                except Exception as e:
                    print("JD",e)
                    continue

                try:
                    other_details = soup.find(class_="other-details")
                    details = other_details.find_all(class_ = "details")
                    for detail in details:
                        data_to_txt_file[detail.label.get_text().upper()] = detail.span.get_text()[:-1]
                except Exception as e:
                    print("OD",e)
                    continue

                try:    
                    education = soup.find(class_="education")
                    education_details = education.find_all(class_ = "details")
                    data_to_txt_file["EDUCATION"] = {}
                    for detail in education_details:
                        data_to_txt_file["EDUCATION"][detail.label.get_text().upper()[:-2]] = detail.span.get_text()
                except:
                    print("ED",e)
                    continue

                try:
                    key_skill = soup.find(class_="key-skill")
                    all_a = key_skill.find_all("a")
                    data_to_txt_file["KEY SKILL"] = []
                    for a in all_a:
                        data_to_txt_file["KEY SKILL"].append(a.get_text())
                except Exception as e:
                    print("KS",e)
                    continue

            except Exception as e:
                print("Main page parser",e)
                continue

            file_name = os.path.join(path, str(file_name_count) + ".json")
            with open(file_name, 'w') as outfile:
                json.dump(data_to_txt_file, outfile)
            driver.close()
            c += 1
            print(file_name_count)
            file_name_count += 1
            driver.switch_to_window(window_before)
            time.sleep(1)
        
        except:
            driver.close()
            c += 1
            driver.switch_to_window(window_before)
            time.sleep(1)
    driver.close()

In [8]:
def scrape_data():
    searchKeyword = "software-developer"
    directory = searchKeyword
    parent_dir = "C:/Users/BASHA/Desktop/"
    path = os.path.join(parent_dir, directory)
    try:
        os.mkdir(path)  
    except FileExistsError:
        print("file exist")
    pagenation_count = 1
    global file_name_count
    file_name_count = 1

    while(file_name_count < 5001):
        if(pagenation_count == 1):
            url = "https://www.naukri.com/{0}-jobs?k={0}".format(searchKeyword)
        else:
            url = "https://www.naukri.com/{0}-jobs-{1}?k={0}".format(searchKeyword,pagenation_count)
        print(url)
        get_url_data(url, path)
        pagenation_count += 1

In [9]:
scrape_data()

https://www.naukri.com/software-developer-jobs?k=software-developer
<ipython-input-7-5bc5c2f55f92>:18: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window_after)
1
<ipython-input-7-5bc5c2f55f92>:70: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window_before)
2
3
4
5
6
7
8
9
10
<ipython-input-7-5bc5c2f55f92>:76: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window_before)
11


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=89.0.4389.90)
